# Using Cholesky and Singular Value Decomposition to generated correlated random numbers
## The problem:
The ability to simulate correlated risk factors is key to many risk models. Historical Simulation achieves this implicitly, by using actual timeseries data for risk factors and applying changes for all risk factors for a given day, for a large number of days (250 or 500 typically). The empirically observed correlations, as well as the means and standard deviations, are implicitly embedded across the historical timeseries data sets.

If we are doing *Monte Carlo* simulation however we need to do something different, since random drawings from a Normal(Gaussian)distribution will be uncorrelated - whereas real data will exhibit correlations. Therefore a technique must be developed to transform uncorrelated random variables to variables which exhibit the empirically observed correlations.

In this Jupyter notebook we explore some techniques for producing correlated random variables and variations on these techniques.
- Cholesky Factorisation : $LL^T=\Sigma$, using both covariance and correlation matrix variations to generate trials 
- Singular Value Decomposition : $UDV^T=\Sigma$ [TODO - help appreciated!]
## Theory - Cholesky Factorisation approach:
Consider a random vector, X, consisting of uncorrelated random variables with each random variable, $X_i$, having zero mean and unit variance 1 ($X\sim N(0,1)$). What we hant is some sort of technique for converting these standard normal variables to correlated variables which exhibit the observed empirical means and variances of theproblem we are modelling.


- Useful identities and results:
    - $\mathbb E[XX^T] = I$, where $X\sim N(0,1)$ Since $Var[XX^T]=\mathbb E [XX^T] + \mathbb E[X] \mathbb E[X^T]$
- To show that we can create new, correlated, random variables $Y$, where $Y=LX$ and
    - $L$ is the Cholesky factorisation matrix (see above "Cholesky"), 
    - X is a vector of independent uncorrelated variables from a Normal distribution with mean of zero and variance of one : $\boxed {X\sim N(0,1)}$
    - $Cov[Y,Y] = \mathbb E[YY^T]


In [4]:
import pandas as pd

In [5]:
from IPython.display import display, Math, Latex, IFrame
import pandas as pd
#import pandas.io.data as pd_io
from pandas_datareader import data, wb
import numpy as np
import scipy as sci
G=pd.DataFrame(np.random.normal(size=(10000000,5)))
m=pd.DataFrame(np.matmul(G.transpose(), G))
display(Math(r'Demonstration~of~~ \mathbb E[XX^T] = I, ~~where~X\sim N(0,1)'))
print(m/10000000)

<IPython.core.display.Math object>

          0         1         2         3         4
0  1.000231 -0.000385  0.000445 -0.000184  0.000035
1 -0.000385  0.999603 -0.000424  0.000493  0.000033
2  0.000445 -0.000424  1.000095  0.000400  0.000417
3 -0.000184  0.000493  0.000400  1.000890 -0.000115
4  0.000035  0.000033  0.000417 -0.000115  0.999320


In [6]:
import pandas as pd
from pandas_datareader import data, wb
import numpy as np
import scipy as sci
stocks=['WDC', 'AAPL', 'IBM', 'MSFT', 'ORCL']
p=data.DataReader(stocks,data_source='google')#[['Adj Close']]
print(type(p))

RemoteDataError: No data fetched using 'GoogleDailyReader'

In [8]:
from pivottablejs import pivot_ui
pivot_ui(m)

In [ ]:
df=p.ix[0]
#df.pop('ATML') get rid of duff entry with NaNs!! - handy as you can just remove (and optionally save) a chunk!!
df=np.log(df/df.shift(1) )
df=df.dropna()
print("Days:{}".format(len(df)))
corr=df.corr()
print(corr)
chol=np.linalg.cholesky(corr)
#chol=sci.linalg.cholesky(corr, lower=True)
print chol
sigma=df.std()
mu=df.mean()
print("sigma=\n{}\n mu=\n{}".format(sigma,mu))
#No generate random normal samples with observed means ("mu"s) and st_devs ("sigma"s)
#G_rands=np.random.normal(loc=mu,scale=sigma,size=(1000,len(sigma)))
G_rands=pd.DataFrame(np.random.normal(size=(1000000,len(sigma))))
#G_Corr_rand=G_rands.dot(chol)
G_Corr_rand=(chol.dot(G_rands.transpose())).transpose()
# Now apply the std dev and mean by multiplation and addition, respectively - return as pandas df
G_=pd.DataFrame(G_Corr_rand * np.broadcast_to(sigma,(1000000,len(sigma))) + np.broadcast_to(mu,(1000000,len(mu))))
print(G_.head())
print(corr)
print(G_.corr())
df.describe().T

In [ ]:
import pandas as pd
from pandas_datareader import data, wb
import numpy as np
import scipy as sci

stocks=['WDC', 'AAPL', 'IBM', 'MSFT', 'ORCL']
p=data.DataReader(stocks,data_source='yahoo')[['Adj Close']]
df=p.ix[0] #convert pandas "panel" to pandas "data frame"
df=np.log(df/df.shift(1) )
df=df.dropna()

cov=df.cov()
chol=np.linalg.cholesky(cov) # default is left/lower; use chol=sci.linalg.cholesky(cov, lower=False) otherwise
print ('Cholesky L=\n{}, \nL^T=\n{},\nLL^T=\n{}'.format(chol, chol.transpose(), chol.dot(chol.T)))

G_rands=pd.DataFrame(np.random.normal(size=(1000000,len(sigma))))
G_=pd.DataFrame((chol.dot(G_rands.transpose())).transpose())

print(G_.head())
print(cov)
print(G_.cov())

In [ ]:
#Check for tiny size - LL^T should be equal to cov, so diff should be negligible
chol.dot(chol.T) - cov

In [ ]:
print (chol.dot(chol.T) - cov).max()